In [229]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import random
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import plotly.express as px


In [258]:
df = pd.read_csv('stats.csv', index_col = 0)

In [259]:
df

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Red Cards,2 Shot Attempts,2 Shots Blocked,2 Total Passes,2 Total Shots on Target,2 Total xG,2 Under Pressure,2 Yellow Cards,winner,fh winner
0,2,1,3,0,1,0.800,0.624,0,2,0,...,0,8,3,175,1,0.876350,0.171,0,2,2
1,2,0,3,0,3,0.903,0.658,0,7,2,...,0,9,6,193,1,0.682699,0.104,0,1,0
2,1,1,6,1,0,0.793,0.468,0,8,4,...,0,5,2,291,0,0.456537,0.096,2,2,0
3,1,2,3,1,0,0.826,0.496,0,5,2,...,0,5,0,304,3,0.162379,0.105,0,1,1
4,3,1,1,0,0,0.784,0.320,0,6,1,...,0,7,0,374,3,0.805794,0.059,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,6,0,1,0,0,0.837,0.628,0,12,3,...,0,0,0,175,0,0.000000,0.206,1,2,0
678,0,4,4,0,1,0.693,0.340,0,0,0,...,0,13,3,316,6,1.904855,0.127,1,2,2
679,2,1,2,1,0,0.571,0.354,0,1,0,...,0,6,2,268,3,0.675959,0.157,0,1,1
680,1,1,3,2,0,0.878,0.665,0,7,1,...,0,2,0,199,1,0.182967,0.131,0,1,1


In [216]:
for index, row in df.iterrows():
    if bool(random.getrandbits(1)):
        row1 = df.iloc[index]
        row1[:15], row1[15:-2] = row1[15:-2], row1[:15].copy()
        row1[-2] = 1 if row1[-2] == 2 else 2 if row1[-2] == 1 else 0
        row1[-1] = 1 if row1[-1] == 2 else 2 if row1[-1] == 1 else 0
        df.iloc[index] = row1

In [147]:
features = df.iloc[: , :-2]
target = df.winner

In [218]:
df.groupby(['winner']).count()[['1 Goals', '2 Goals','1 Possesion','2 Possesion']]

,1 Goals,2 Goals,1 Possesion,2 Possesion
winner,,,,
0,120,120,120,120
1,275,275,275,275
2,287,287,287,287


In [215]:
df.groupby(['winner']).count()[['1 Goals', '2 Goals','1 Possesion','2 Possesion']]

,1 Goals,2 Goals,1 Possesion,2 Possesion
winner,,,,
0,120,120,120,120
1,313,313,313,313
2,249,249,249,249


In [219]:
X_train, X_test, y_train, y_test = train_test_split(features, target)

In [220]:
cl_estimators = {
    'k-Nearest Neighbor': KNeighborsClassifier(), 
    'Gaussian Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(max_iter = 100000)}

In [221]:
def classifiers_models():
    for name, model in cl_estimators.items(): 
        x = model.fit(X=X_train, y = y_train)
        accuracytest = x.score(X_test, y_test)
        accuracytrain = x.score(X_train, y_train)
        print(name + ": \n\t" + "Classification accuracy on the test data:", f"{accuracytest:.2%}")
        print(f"\tClassification accuracy on the training data: {accuracytrain:.2%}\n")

In [222]:
classifiers_models()

k-Nearest Neighbor: 
	Classification accuracy on the test data: 57.31%
	Classification accuracy on the training data: 72.99%

Gaussian Naive Bayes: 
	Classification accuracy on the test data: 59.65%
	Classification accuracy on the training data: 69.08%

Decision Tree: 
	Classification accuracy on the test data: 58.48%
	Classification accuracy on the training data: 100.00%

Logistic Regression: 
	Classification accuracy on the test data: 73.10%
	Classification accuracy on the training data: 74.17%



In [223]:
def cl_model_comp(features, target):
    for name, model in cl_estimators.items():
      kfold = StratifiedKFold(n_splits = 10, random_state = 300, shuffle = True)
      scores = cross_val_score(model, X=features,y=target,cv=kfold)
      print(f'{name}: \n\t mean accuracy={scores.mean():.2%}, standard deviation={scores.std():.2%}\n')

In [234]:
cl_model_comp(features, target)

k-Nearest Neighbor: 
	 mean accuracy=61.71%, standard deviation=4.79%

Gaussian Naive Bayes: 
	 mean accuracy=62.90%, standard deviation=4.60%

Decision Tree: 
	 mean accuracy=61.89%, standard deviation=4.73%

Logistic Regression: 
	 mean accuracy=71.55%, standard deviation=3.20%



In [236]:
def std_scaler(features):
    """
    Normalizes a dataset using a standard scaler (mean of 0, standard deviation of 1 for each feature)
    """
    
    scaler = StandardScaler()
    scaler.fit(features)
    
    features_scaled = pd.DataFrame(scaler.transform(features), columns = features.columns)
    
    return features_scaled

In [237]:
from sklearn.preprocessing import StandardScaler


X_train_scaled_cl = std_scaler(X_train)
X_test_scaled_cl = std_scaler(X_test)

In [240]:
def classifiers_models_scaled():
    for name, model in cl_estimators.items(): 
        x = model.fit(X=X_train_scaled_cl, y = y_train)
        accuracytest = x.score(X_test_scaled_cl, y_test)
        accuracytrain = x.score(X_train_scaled_cl, y_train)
        print(name + ": \n\t" + "Classification accuracy on the test data:", f"{accuracytest:.2%}")
        print(f"\tClassification accuracy on the training data: {accuracytrain:.2%}\n")

In [233]:
pca = PCA(n_components = 2, random_state = 3000)
reduced_data = pca.fit_transform(features)
reduced_df = pd.DataFrame(reduced_data, columns = ["Component 1", "Component 2"])
reduced_df['position'] = target.reset_index().drop('index', axis=1)

graph = px.scatter(reduced_df, x="Component 1", y="Component 2", color='position')
graph.show()

In [241]:
classifiers_models_scaled()

k-Nearest Neighbor: 
	Classification accuracy on the test data: 64.33%
	Classification accuracy on the training data: 75.73%

Gaussian Naive Bayes: 
	Classification accuracy on the test data: 47.37%
	Classification accuracy on the training data: 54.99%

Decision Tree: 
	Classification accuracy on the test data: 58.48%
	Classification accuracy on the training data: 100.00%

Logistic Regression: 
	Classification accuracy on the test data: 71.35%
	Classification accuracy on the training data: 75.15%



In [187]:
def logistic_tuning():
    param_grid = {"solver":["sag", "saga","liblinear","newton-cg", "lbfgs"]}
    grid_search = GridSearchCV(LogisticRegression(multi_class='multinomial', max_iter = 100000), param_grid, cv=10)
    grid_search.fit(X=X_train, y=y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Training set score with best parameter: ", grid_search.score(X_train, y_train))
    print("Test set score with best parameter: ", grid_search.score(X_test, y_test))

In [188]:
logistic_tuning()

C:\Users\idotz\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\idotz\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\idotz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1350, in fit
    multi_class = _check_multi_class(self.multi_class, solver,
  File "C:\Users\idotz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 473, in _check_multi_class
    raise ValueError("Solver %s does not support "
ValueError: Solver liblinear does not support a multinomial backend.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\idotz\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarni

C:\Users\idotz\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.67911011 0.68299397        nan 0.68303167 0.68499246]
  warnings.warn(


Best parameters:  {'solver': 'lbfgs'}
Training set score with best parameter:  0.7240704500978473
Test set score with best parameter:  0.7368421052631579


In [244]:
model_cl = LogisticRegression(multi_class='multinomial', solver = "newton-cg",max_iter=100000)
model_cl.fit(X=X_train, y = y_train)
predicted_cl = model_cl.predict(X=features)
pred_test = model_cl.predict(X=X_test)

In [245]:
odds = model_cl.predict_proba(X = features)
oddf = pd.DataFrame(odds, columns = ['% Tie', '% 1 Win', '% 2 Win'])

In [246]:
df['win prediction'] = predicted_cl
df = pd.concat([df, oddf.reindex(df.index)], axis=1)

In [195]:
df[(df['winner'] != df['win prediction']) & (df['winner'] != df['fh winner'])]

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Yellow Cards,winner,fh winner,win prediction,% Tie,% 1 Win,% 2 Win,% Tie,% 1 Win,% 2 Win
2,1,1,6,1,0,0.793,0.468,0,8,4,...,2,2,0,1,0.077126,0.585915,0.336958,0.134026,0.529638,0.336336
5,2,2,5,0,0,0.803,0.620,0,2,0,...,1,1,2,2,0.208304,0.317535,0.474161,0.305488,0.260402,0.434110
7,8,2,2,0,0,0.838,0.596,0,15,5,...,0,2,0,0,0.420329,0.390421,0.189251,0.441117,0.372199,0.186684
12,2,1,7,3,4,0.770,0.506,0,7,3,...,0,2,1,1,0.048589,0.941042,0.010369,0.015297,0.981959,0.002744
14,0,1,1,0,1,0.739,0.462,0,4,1,...,0,2,0,0,0.357811,0.291216,0.350973,0.365071,0.287494,0.347435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,2,0,1,1,2,0.828,0.366,0,3,1,...,0,0,1,1,0.239313,0.577211,0.183477,0.271618,0.582706,0.145676
662,5,1,0,1,0,0.876,0.786,0,6,1,...,1,0,1,1,0.220562,0.763582,0.015856,0.186503,0.799566,0.013931
663,2,1,3,0,0,0.800,0.489,0,4,0,...,0,0,2,2,0.341748,0.112296,0.545956,0.428690,0.116816,0.454493
676,0,1,4,0,0,0.732,0.333,0,1,0,...,1,2,0,1,0.202538,0.393141,0.404321,0.224613,0.402498,0.372889


In [247]:
oddf = pd.DataFrame(odds, columns = ['% Tie', '% 1 Win', '% 2 Win'])

In [200]:
print(accuracy_score(y_test,pred_test))

0.7368421052631579


In [250]:
df.iloc[46]

1 Corners                    1.000000
1 Fouls Committed            1.000000
1 Fouls Wons                 0.000000
1 Goals                      1.000000
1 Offsides                   1.000000
1 Pass Completion            0.679000
1 Possesion                  0.344000
1 Red Cards                  0.000000
1 Shot Attempts              1.000000
1 Shots Blocked              0.000000
1 Total Passes             131.000000
1 Total Shots on Target      0.000000
1 Total xG                   0.760000
1 Under Pressure             0.198000
1 Yellow Cards               1.000000
2 Corners                    4.000000
2 Fouls Committed            1.000000
2 Fouls Wons                 4.000000
2 Goals                      1.000000
2 Offsides                   0.000000
2 Pass Completion            0.824000
2 Possesion                  0.656000
2 Red Cards                  0.000000
2 Shot Attempts              7.000000
2 Shots Blocked              2.000000
2 Total Passes             250.000000
2 Total Shot

In [185]:
mt = []
for f, i in zip(df.columns[:-6],model_cl.coef_[1]):
    mt.append((f, i))

In [167]:
df = pd.concat([df, oddf.reindex(df.index)], axis=1)

In [257]:
df.iloc[17]

1 Corners                    2.000000
1 Fouls Committed            2.000000
1 Fouls Wons                 2.000000
1 Goals                      0.000000
1 Offsides                   0.000000
1 Pass Completion            0.783000
1 Possesion                  0.378000
1 Red Cards                  0.000000
1 Shot Attempts              3.000000
1 Shots Blocked              0.000000
1 Total Passes             203.000000
1 Total Shots on Target      0.000000
1 Total xG                   0.085754
1 Under Pressure             0.246000
1 Yellow Cards               2.000000
2 Corners                    7.000000
2 Fouls Committed            0.000000
2 Fouls Wons                 4.000000
2 Goals                      0.000000
2 Offsides                   0.000000
2 Pass Completion            0.835000
2 Possesion                  0.622000
2 Red Cards                  0.000000
2 Shot Attempts              9.000000
2 Shots Blocked              0.000000
2 Total Passes             334.000000
2 Total Shot

In [251]:
df.head(50)

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Total Shots on Target,2 Total xG,2 Under Pressure,2 Yellow Cards,winner,fh winner,win prediction,% Tie,% 1 Win,% 2 Win
0,6,2,0,1,1,0.651,0.376,0,8,3,...,0,0.069639,0.279,0,1,1,2,0.150846,0.160053,0.689101
1,2,0,3,0,3,0.903,0.658,0,7,2,...,1,0.682699,0.104,0,1,0,1,0.199574,0.584739,0.215686
2,0,1,5,1,0,0.890,0.532,0,5,2,...,3,0.209009,0.070,1,1,0,1,0.243927,0.460717,0.295357
3,1,1,5,0,1,0.803,0.504,0,5,0,...,1,0.884572,0.110,1,2,2,1,0.311764,0.604139,0.084097
4,3,1,1,0,0,0.784,0.320,0,6,1,...,3,0.805794,0.059,0,2,2,2,0.123613,0.020155,0.856232
5,2,2,5,0,0,0.803,0.620,0,2,0,...,0,0.397853,0.074,1,1,2,2,0.320896,0.279314,0.399790
6,5,1,3,0,0,0.687,0.366,0,7,1,...,3,0.888606,0.122,1,2,0,2,0.304089,0.236439,0.459472
7,8,2,2,0,0,0.838,0.596,0,15,5,...,1,0.083115,0.180,0,2,0,1,0.321358,0.468012,0.210630
8,0,0,2,1,0,0.712,0.333,0,7,1,...,1,0.430447,0.110,1,1,1,2,0.240371,0.197957,0.561672
9,0,0,1,1,4,0.709,0.312,0,2,1,...,2,1.006015,0.053,0,2,0,1,0.133431,0.833081,0.033488


In [242]:
df

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Red Cards,2 Shot Attempts,2 Shots Blocked,2 Total Passes,2 Total Shots on Target,2 Total xG,2 Under Pressure,2 Yellow Cards,winner,fh winner
0,6,2,0,1,1,0.651,0.376,0,8,3,...,0,2,0,290,0,0.069639,0.279,0,1,1
1,2,0,3,0,3,0.903,0.658,0,7,2,...,0,9,6,193,1,0.682699,0.104,0,1,0
2,0,1,5,1,0,0.890,0.532,0,5,2,...,0,8,4,256,3,0.209009,0.070,1,1,0
3,1,1,5,0,1,0.803,0.504,0,5,0,...,0,5,2,299,1,0.884572,0.110,1,2,2
4,3,1,1,0,0,0.784,0.320,0,6,1,...,0,7,0,374,3,0.805794,0.059,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,0,0,1,0,0,0.703,0.372,0,0,0,...,0,12,3,295,4,0.593374,0.247,0,1,0
678,4,1,5,1,0,0.873,0.660,0,13,3,...,0,0,0,163,0,0.000000,0.178,0,1,1
679,8,0,7,0,0,0.720,0.646,0,6,2,...,0,1,0,147,0,0.215750,0.170,1,2,2
680,2,1,1,0,0,0.734,0.335,0,2,0,...,0,7,1,395,1,1.371691,0.089,0,2,2
